# Connect to Drive & Read

In [1]:
from google.colab import drive
import pandas as pd, numpy as np
from plotly import express as px
#  pip install -U kaleido

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df = pd.read_pickle("/content/drive/MyDrive/CMG - Crystal Prediction Project/Ternary Materials Point Group Prediction/Data/NOMAD_2/Classification_Data_8.pkl").dropna()
print(df.shape)
df.head()

(1381099, 24)


,Atomic Number 1,Atomic Number 2,Atomic Number 3,Element_1,Element_2,Element_3,Coefficient 1,Coefficient 2,Coefficient 3,formula_reduced,bravais_lattice,crystal_system,space_group_number,point_group,lattice_parameters,Oxidation 1,Oxidation 2,Oxidation 3,IonicRadius_1,IonicRadius_2,IonicRadius_3,IonizationPot1st_1,IonizationPot1st_2,IonizationPot1st_3
0,3,5,41,Li,B,Nb,4,1,1,BLi4Nb,cF,cubic,216,-43m,"{'a': 6.71860936e-10, 'b': 6.71860936e-10, 'c'...",1,-1,-3,0.755440,1.048943,1.842691,5.3917,8.298,6.7589
1,4,5,41,Be,B,Nb,4,1,1,BBe4Nb,cF,cubic,216,-43m,"{'a': 6.119642980000002e-10, 'b': 6.1196429800...",1,-1,-3,0.643658,1.048943,1.842691,9.3226,8.298,6.7589
2,4,5,41,Be,B,Nb,2,1,1,BBe2Nb,cF,cubic,216,-43m,"{'a': 5.458027e-10, 'b': 5.458027e-10, 'c': 5....",2,-1,-3,0.452080,1.048943,1.842691,9.3226,8.298,6.7589
3,4,5,41,Be,B,Nb,2,1,1,BBe2Nb,cF,cubic,225,m-3m,"{'a': 5.420156379999999e-10, 'b': 5.4201563799...",2,-1,-3,0.452080,1.048943,1.842691,9.3226,8.298,6.7589
4,4,5,41,Be,B,Nb,2,1,1,BBe2Nb,mS,monoclinic,12,2/m,"{'a': 4.660283620000524e-10, 'b': 8.41781306e-...",2,-1,-3,0.452080,1.048943,1.842691,9.3226,8.298,6.7589


**Config**

In [3]:
FEATURE_NAMES = [
#     "Atomic Number 1","Atomic Number 2","Atomic Number 3",
    "Coefficient 1", "Coefficient 2", "Coefficient 3", 
    "IonizationPot1st_1", "IonizationPot1st_2", "IonizationPot1st_3", 
    "Oxidation 1", "Oxidation 2", "Oxidation 3", # Used to be electronegativity 
    "IonicRadius_1", "IonicRadius_2", "IonicRadius_3"
]

GROUPERS = [
    "Atomic Number 1","Atomic Number 2","Atomic Number 3",
    "Coefficient 1", "Coefficient 2", "Coefficient 3"
]

# Y_NAME = 'crystal_system'
# Y_NAME = 'bravais_lattice'
# Y_NAME = 'crystal_system'
Y_NAME = 'space_group_number'

# Preprocess

In [4]:
df['crystal_system'].replace(to_replace="trigonal", value="hexagonal", inplace=True)

In [5]:
df_dedup = df[FEATURE_NAMES+[Y_NAME]].drop_duplicates().copy()
df_dedup[Y_NAME] = df_dedup[Y_NAME].astype(int)
df_dedup[Y_NAME].unique().shape

(207,)

In [6]:
value_counts = df_dedup[Y_NAME].value_counts()
value_counts.index = value_counts.index.astype(str)
fig = px.bar(x=value_counts.index, y=value_counts.values)
fig.update_layout( 
    width=3000,
    yaxis=dict(title='Count'),
    xaxis=dict(
        title='Space Group',
        tickmode='linear',
        tickangle=60,
        type='category'
    )
)

# Base Experiment

1. Resample
2. Groupby (Multi-Labeled data)
3. Multi-Label Encoding
4. Train-Test Split
5. Train

In [12]:
accepted_classes = (df_dedup[Y_NAME].value_counts() > 300).replace(False, np.nan).dropna().index.tolist()
df_filtered = df_dedup[df_dedup[Y_NAME].isin(accepted_classes)].copy()
df_filtered[Y_NAME] = df_filtered[Y_NAME].astype(int)

print("Previous Data Size:", df_dedup.shape[0])
print("New Data Size:", df_filtered.shape[0])
print("Number of Classes Left:", len(accepted_classes))

Previous Data Size: 1381099
New Data Size: 1372271
Number of Classes Left: 50


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

df_grouped2 = df_filtered.groupby(FEATURE_NAMES)[Y_NAME].apply(list).reset_index()

In [14]:
ml_binner = MultiLabelBinarizer()
ml_binner.fit(df_grouped2[Y_NAME])
y = ml_binner.transform(df_grouped2[Y_NAME])
X = df_grouped2[FEATURE_NAMES].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [15]:
clf = RandomForestClassifier(n_estimators=50, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=50, random_state=0)

In [16]:
y_pred = clf.predict(X_test)

In [17]:
from sklearn.metrics import classification_report
target_names = ml_binner.classes_
print(classification_report(y_test, y_pred, target_names=[str(i) for i in target_names]))

              precision    recall  f1-score   support

           1       0.66      0.23      0.34       223
           2       0.68      0.26      0.37       275
           4       0.80      0.17      0.29        69
           6       0.81      0.89      0.85     10047
           8       0.91      0.96      0.93     12177
          10       0.93      0.98      0.96     12553
          11       0.74      0.23      0.35       114
          12       0.91      0.96      0.93     12619
          14       0.70      0.30      0.42       427
          15       0.74      0.40      0.52       290
          25       0.76      0.73      0.74      7246
          33       0.62      0.09      0.16        54
          35       0.83      0.91      0.87     10220
          36       0.72      0.36      0.48        73
          38       0.66      0.41      0.51      3007
          42       0.83      0.91      0.87      3506
          44       0.68      0.76      0.71     11098
          47       0.91    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.



# Iterations

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, balanced_accuracy_score, matthews_corrcoef

def prepare_train_test(df):
    ml_binner = MultiLabelBinarizer()
    ml_binner.fit(df[Y_NAME])
    y = ml_binner.transform(df[Y_NAME])
    X = df[FEATURE_NAMES].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    return X_train, X_test, y_train, y_test, ml_binner

def evaluate(y_true, y_pred):
    micro_f1 = f1_score(y_true, y_pred, average='micro')
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    WBA, WMCC = calc_wba_wmcc(y_true, y_pred)
    metrics = {
        "WMCC": WMCC ,
        "micro_f1": micro_f1,
        "macro_f1": macro_f1,
        "WBA": WBA,
    }
    return metrics

def calc_wba_wmcc(y_true, y_pred):
    """Calculated the weighted balanced accuracy and mathews correlation coefficient
    """
    n_lst = []
    ba_lst = []
    mcc_lst = []
    for idx in range(y_true.shape[1]):
        class_true = y_true[:, idx]
        class_preds = y_pred[:, idx]
        N = sum(class_true)
        BA = balanced_accuracy_score(class_true, class_preds, adjusted=False)
        MCC = matthews_corrcoef(class_true, class_preds)
        n_lst.append(N)
        mcc_lst.append(MCC)
        ba_lst.append(BA)

    weighted_balanced_acc = sum([n*ba for n,ba in zip(n_lst,ba_lst)]) / sum(sum(y_true))
    weighted_mcc = sum([n*mcc for n,mcc in zip(n_lst, mcc_lst)]) / sum(sum(y_true))

    return weighted_balanced_acc, weighted_mcc


def filter_y(df, min_class_size):
    accepted_classes = (df[Y_NAME].value_counts() > min_class_size).replace(False, np.nan).dropna().index.tolist()
    df_filtered = df[df[Y_NAME].isin(accepted_classes)].copy()
    return df_filtered

In [8]:
df_main = df_dedup.copy()

In [15]:
experiments = []

2**16

65536

In [162]:
for power in range(16, 17):
    threshold = 2**power
    
    # Filter
    df_filtered = filter_y(df_main, threshold)

    # Group
    df_agg = df_filtered.groupby(FEATURE_NAMES)[Y_NAME].apply(list).reset_index()

    # Prepare
    X_train, X_test, y_train, y_test, ml_binner = prepare_train_test(df_agg)

    # Learning
    clf = RandomForestClassifier(n_estimators=50, random_state=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Evaluate
    eval_dict = evaluate(y_test, y_pred)

    n_classes = df_filtered[Y_NAME].unique().shape[0]
    data_size = df_agg.shape[0]

    exp = {
        "n_classes": n_classes,
        "thresh": threshold,
        "data_size": data_size,
        "evaluation": eval_dict
    }
    experiments.append(exp)

    for key,val in exp.items():
        print(key, ":", val)
        print("-"*20)

n_classes : 7
--------------------
thresh : 65536
--------------------
data_size : 297721
--------------------
evaluation : {'WMCC': 0.9937754088961148, 'micro_f1': 0.9954043319304174, 'macro_f1': 0.99425133340205, 'WBA': 0.9973861434220035}
--------------------


In [25]:
eval_dict

{'WBA': 0.9585578662076325,
 'WMCC': 0.9093312823050367,
 'macro_f1': 0.8633444877784667,
 'micro_f1': 0.9250762303121532}

In [168]:

import pickle
with open('/content/drive/MyDrive/experiments_res2.pkl', 'wb') as f:
    pickle.dump(exps, f)


In [163]:

with open('/content/drive/MyDrive/experiments_res.pkl', 'rb') as f:
    exps = pickle.load(f)


In [167]:
exps.append(experiments[1])

In [161]:
with open('/content/drive/MyDrive/experiments_res2.pkl', 'rb') as f:
    x = pickle.load(f)
    print(x)

[2, 4, 3, 6, 1, 7]


,WMCC,micro_f1,macro_f1,WBA
0,0.897911,0.915582,0.326555,0.951166
1,0.898046,0.915526,0.356526,0.951254
2,0.897916,0.915549,0.352407,0.951077
3,0.898592,0.916026,0.392493,0.951450
4,0.898815,0.916228,0.448186,0.951588
5,0.900430,0.917252,0.505659,0.952501
6,0.899939,0.917087,0.528533,0.952157
7,0.899948,0.917128,0.569235,0.952342
8,0.900576,0.917565,0.639828,0.952736
9,0.903142,0.919611,0.717317,0.954335


In [177]:
pd.DataFrame(exps).join(pd.DataFrame([exp['evaluation'] for exp in exps])).drop("evaluation", axis=1).to_csv("eval.csv")

In [27]:
wba_lst = [elem['evaluation']['WBA'] for elem in x]
macro_f_lst = [elem['evaluation']['macro_f1'] for elem in x]
micro_f_lst = [elem['evaluation']['micro_f1'] for elem in x]
wmcc_lst = [elem['evaluation']['WMCC'] for elem in x]
class_size_lst = [elem['n_classes'] for elem in x]
thresh_lst = [elem['thresh'] for elem in x]
data_size_lst = [elem['data_size'] for elem in x]

In [122]:
# wmcc_lst

# [0.5*(1+mcc) for mcc in wmcc_lst]

In [155]:
pd.DataFrame(x)

,0
0,2
1,4
2,3
3,6
4,1
5,7


In [149]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

fig, ax = plt.subplots( nrows=1, ncols=1, figsize=(14, 8), dpi=200) # create figure & 1 axis


ax.plot(class_size_lst[:20], wba_lst[:20], label = 'WBA',  marker='o', linestyle='--', linewidth=2)
ax.plot(class_size_lst[:20], macro_f_lst[:20], label = 'Macro F1',  marker='o', linestyle='--', linewidth=2)
ax.plot(class_size_lst[:20], micro_f_lst[:20], label = 'Micro F1',  marker='o', linestyle='--', linewidth=2)
ax.plot(class_size_lst[:20], wmcc_lst[:20], label = 'WMCC',  marker='o', linestyle='--', linewidth=2)

      
plt.xlabel('Number of Classes', fontsize=18)
plt.ylabel('Score', fontsize=18)
plt.legend(loc='lower right')
plt.title('Class Size Threshold Affect on Classification Performance', fontsize=22, y=1.05)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.legend(fontsize=20) # using a size in points
plt.legend(fontsize="x-large")
fig.savefig('thresh_affect_class_change.png')   # save the figure to file
plt.close(fig)    # close the figure window
# plt.show()

In [150]:
# plt.savefig("thresh_affect_class_change.png")
# plt.close(fig)    # close the figure window

In [151]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

# figure(figsize=(14, 8), dpi=80)
fig, ax = plt.subplots( nrows=1, ncols=1, figsize=(14, 8), dpi=200) # create figure & 1 axis



ax.plot(thresh_lst[:20], wba_lst[:20], label = 'WBA',  marker='o', linestyle='--', linewidth=2)
ax.plot(thresh_lst[:20], macro_f_lst[:20], label = 'Macro F1',  marker='o', linestyle='--', linewidth=2)
ax.plot(thresh_lst[:20], micro_f_lst[:20], label = 'Micro F1',  marker='o', linestyle='--', linewidth=2)
ax.plot(thresh_lst[:20], wmcc_lst[:20], label = 'WMCC',  marker='o', linestyle='--', linewidth=2)

      
plt.xlabel('Class Size Threshold', fontsize=18)
plt.ylabel('Score', fontsize=18)
plt.legend(loc='lower right')
plt.title('Class Size Threshold Affect on Classification Performance', fontsize=22, y=1.05)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
ax.set_xscale('log', basex=2)

plt.legend(fontsize=20) # using a size in points
plt.legend(fontsize="x-large")
plt.savefig("thresh_affect_thresh_change.png")
plt.close(fig)    # close the figure window
# plt.show()

In [129]:


# import matplotlib.pyplot as plt
# from matplotlib.pyplot import figure

# figure(figsize=(14, 8), dpi=80)


# plt.plot(data_size_lst[:20], wba_lst[:20], label = 'WBA',  marker='o', linestyle='--', linewidth=2)
# plt.plot(data_size_lst[:20], macro_f_lst[:20], label = 'Macro F1',  marker='o', linestyle='--', linewidth=2)
# plt.plot(data_size_lst[:20], micro_f_lst[:20], label = 'Micro F1',  marker='o', linestyle='--', linewidth=2)
# plt.plot(data_size_lst[:20], wmcc_lst[:20], label = 'WMCC',  marker='o', linestyle='--', linewidth=2)

      
# plt.xlabel('Class Size Threshold', fontsize=18)
# plt.ylabel('Score', fontsize=18)
# plt.legend(loc='lower right')
# plt.title('Class Size Threshold Affect on Classification Performance', fontsize=22, y=1.05)
# plt.xticks(fontsize=14)
# plt.yticks(fontsize=14)



# plt.legend(fontsize=20) # using a size in points
# plt.legend(fontsize="x-large")
# plt.gca().invert_xaxis()
# plt.show() 

In [130]:


# import matplotlib.pyplot as plt
# from matplotlib.pyplot import figure

# fig, ax = plt.subplots(1,1, figsize=(14,8))



# ax.plot(data_size_lst[:20], wba_lst[:20], label = 'WBA',  marker='o', linestyle='--', linewidth=2)
# ax.plot(data_size_lst[:20], macro_f_lst[:20], label = 'Macro F1',  marker='o', linestyle='--', linewidth=2)
# ax.plot(data_size_lst[:20], micro_f_lst[:20], label = 'Micro F1',  marker='o', linestyle='--', linewidth=2)
# ax.plot(data_size_lst[:20], wmcc_lst[:20], label = 'WMCC',  marker='o', linestyle='--', linewidth=2)

      
# plt.xlabel('Class Size Threshold', fontsize=18)
# plt.ylabel('Score', fontsize=18)
# ax.legend(loc='lower right')
# plt.title('Class Size Threshold Affect on Classification Performance', fontsize=22, y=1.05)
# plt.xticks(fontsize=14)
# plt.yticks(fontsize=14)
# ax.set_xticklabels([str(i) for i in data_size_lst[:20]])


# plt.legend(fontsize=20) # using a size in points
# plt.legend(fontsize="x-large")
# plt.gca().invert_xaxis()
# plt.show()